In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

# daily rainfall
rain = pd.read_csv('assets/rainfall_daily.csv')
rain = rain.set_index('Date')

# surface water flow rate
flow = pd.read_csv('assets/flowrate.csv')
flow['Date']=flow['Datetime'].str.split('T').str[0]
flow = flow[['Date', 'Flow_rate']]
flow= flow.set_index('Date')

#nitrate
no3 = pd.read_csv('assets/nitrate.csv')
no3['Date']=no3['Datetime'].str.split('T').str[0]
no3 = no3 [['Date', 'Nitrate']]
no3 = no3.set_index('Date')

# initial visualization
fig, ax = plt.subplots(3,1, figsize= (12, 8))
rain.plot(ax=ax[0], title = 'Daily Rainfall (mm)')
flow.plot(ax=ax[1], title = 'Flow Rate (L/s)')
no3.plot(ax=ax[2], title = 'Nitrate Concentration (mg/L)')
plt.tight_layout()
plt.savefig('Initial_visualization.pdf')

# for Correlation analysis
# rainfall
rain = rain.reset_index()
rain['Date'] = pd.to_datetime(rain['Date'])

# flow rate
flow = flow.reset_index()
flow['Date'] = pd.to_datetime(flow['Date'])

#nitrate
no3 = no3.reset_index()
no3['Date'] = pd.to_datetime(no3['Date'])

# merging data on "Date"
df = rain.merge(flow, on='Date', how='inner').merge(no3, on='Date', how='inner')
df = df.dropna()
df = df.set_index('Date')

#correlation
print("Correlation Matrix:\n", df.corr())
correl_matrix = df.corr()
correl_matrix.to_excel('correl_matrix.xlsx', sheet_name='correl')

# Nutrient load model: load= concentration X flow 
df['nit_load_kg_per_day'] = df['Nitrate']*df['Flow_rate']*0.0864

# plotting Nitrate load
plt.figure(figsize= (10,4))
df['nit_load_kg_per_day'].plot(title='Daily Nitrate Load (kg/day)')
plt.ylabel('Nitrate Load (kg/day)')
plt.tight_layout()
plt.savefig('Nitrate_load.pdf');

fig, ax = plt.subplots(2,1, figsize=(12,8), sharex=True)

#plot 1: Rainfall

ax[0].plot(df.index, df['Rainfall_total'], color='royalblue')
ax[0].set_ylabel('Rainfall (mm)', color='royalblue')
ax[0].set_title('Daily Rainfall')

#plot 2: Nitrate load
ax[1].plot(df.index, df['nit_load_kg_per_day'], color='darkgreen')
ax[1].set_ylabel('Nitrate Load (kg/day)', color='darkgreen')
ax[1].set_title('Daily Nitrate Load')

plt.tight_layout()
plt.savefig('Nitrate_Vs_Rainfall.pdf')
plt.show()